In [3]:
#1. PCGP 
import os


#(1). PCGP RNA ITD from sqlite table
# sqlite db table location /research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/PCGP/cnv-sv/fusion
# script /research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/PCGP/cnv-sv/ITD_PCGP_EX.py
# PCGP samples /research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/PCGP/mds.snvindel
# This script will generate itd.bedj and move it to /research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/PCGP/cnv-sv/itd

os.chdir('/research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/PCGP/cnv-sv/')
!python3 ITD_PCGP_EX.py fusion /research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/PCGP/mds.snvindel/PCGP_SAMPLES itd.bedj
!mv itd.bedj ../itd



#(2). PCGP svcnvloh

os.chdir('/research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/PCGP/cnv-sv/')

"""
#SVCNV from PCGP papers
#Generate PCGP_svcnv.bedj

print('(1). PCGP individual paper...')

#(I). 2011_ETP_TALL 22237106
# S2(SJTALLcnv_snp6) consider later
#os.chdir('2011_ETP_TALL')
#!python3 2011_ETP_TALL_svcnv.py --cnv  SJTALL_DNA_CNA_snparray.hg18 --sample /home/jwang7/tp/hg19/PCGP/mds.snvindel/2011_ETP_TALL/2011_ETP_TALL_SAMPLE
#print('\t2011_ETP_TALL done')

#(II). 2014_EWS 25223734
#All included

#(III). 2013_E_RHB 24332024
# fusion only, no CNV, SV included
os.chdir('2013_E_RHB')
!python3 2013_E_RHB_svcnv.py --fusion 2013_E_RHB_fusion
print('\t2013_E_RHB done')

#(IV). 2014_Ph-like_ALL 25207766
#No CNV, SV
#fusion no genomic position

#(V). 2012_hypodiploid_ALL 23334668
os.chdir('../')
!python3 2012_hypodiploid_ALL_svcnv.py CNV MISSED_CNVID cnvsv
print('\t2012_hypodiploid_ALL done')

#(VI). 2015_MEL 25268584
"""

#1'. Sith CNV
# take raw CNV file (cnv_analysis_l2r_pre_2017-09-25_14_28_12.txt) as input
# generate cnv.raw
# remove CNVs with invalid pos or duplicated CNVS
# remove CNVs from chr8 of three samples including SJBALL020877_D1, SJBALL020013_D1, SJBALL020625_D1

!python3 cnv_cleanup.py -f raw/cnv_analysis_l2r_pre_2017-09-25_14_28_12.txt -o cnv.raw

#2'. Sith SV
!node sv.dedup.js >sv.raw

#3'. meger cnv.raw, sv,raw
#modified parse.svCNV.2.js, original one parse.svCNV.2.js_Deprecated 
#do not add PCGP_LOH at this step. will add PCGP_WGS_LOH(PCGP_LOH here) and PCGP_WES_CNVLOH later
!node parse.svCNV.2.js > svcnvloh

#4'. PCGP WGS LOH pcgpWgs.loh
#from /home/jwang7/Work/TARGET_WES_CONSERTING/VCF2CNA-master
!cat pcgpWgs.loh >> svcnvloh

#5'. PCGP WES CNV and LOH pcgpWes.cnvloh
#from /home/jwang7/Work/PCGP_CONSERTING
!cat pcgpWes.cnvloh >> svcnvloh

#6'. add CNV and SV from PCGP paper
#annotate CNV,SV and LOH(add pmid)
!python3 AttrUpdate_NewMuaAdd_CNVSV.py --oldFile svcnvloh --CNV ALLCNVs --SV ALLSVs --samplearray ALLSAMPLES -o svcnvloh.annotated




#(3). PCGP RNA fusion

#1'. PCGP_fusion
!python3 pcgp_fusion.py fusion PCGP_fusion

#2'. annotate RNA fusion
!python3 annotate_fusion.py ALLSAMPLES PCGP_fusion fusion_PCGPpaper PCGP_fusion.annotated

5 CNVs with invalid position were removed!
4001 duplicated CNVs were removed!


In [6]:
#2. TARGET
#Contain only diagnosis tumors
#dna_assay: wes
#project: pantarget
#vorigin: somatic
#pmid: 29489755


import os

os.chdir('/research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/TARGET/cnv-sv/')

##################0-based or 1-based ??

#(1). WES CNV from Jian, Has relapse, hg19/TARGET/cnv-sv/raw/wes.conserting.cnv
#CGI CNV from Xiaotu, Has relapse hg19/TARGET/cnv-sv/raw/CNV_every_case.txt
#CNV snp6 TALL, Use all samples, hg19/TARGET/cnv-sv/raw/TARGET_T_260pd_CBS_cnvFiltered_DGV_merged_hg19_IGV_w_genes.txt
#Keep only data for diagnosis samples by checking with "target.samples"

!node changename.cnv.js > svcnv


#(2). CGI LOH from xiaotu, Has relapse, hg19/TARGET/cnv-sv/raw/LOH_every_case.txt

!node changename.loh.js >> svcnv


#(3). CGI SV, 0-based, Diagnosis only, hg19/TARGET/cnv-sv/raw/SV_table.txt

!node changename.sv.js >> svcnv


#(4). RNA fusion
#PanTarget/CGI_WXS_D/Fusion.txt
#Following files have relapse samples
#hg19/TARGET/cnv-sv/raw/NBL_final_fusions_manualReview_good.YLi
#hg19/TARGET/cnv-sv/raw/WT_final_fusions_manualReview_goodones.YLi
#For sample OS_diagnosis_PAPFLB, drop 5 of its TP53 fusion, keep only 2

!node changename.fusion.js > fusion


#(5). TARGET ITD

os.chdir('/research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/TARGET/itd/')
!python3 ITD_TARGET_EX.py -f ITD_TARGET_table --extfile TARITD2 -m ../../../anno/db/target.samples --DiaSample D6SJIDTARGETdiagnosis -o itd.bedj



target CNV: 201 samples wrong: TARGET-10-PAKKCA-03A-01D TARGET-10-PALIYC-03A-01D TARGET-10-PANCVR-04A-01D TARGET-10-PANDWE-04A-01D TARGET-10-PANKAK-04A-01D TARGET-10-PANKMB-03A-01D TARGET-10-PANSDA-04A-01D TARGET-10-PANSHK-04A-01D TARGET-10-PANUKN-04A-01D TARGET-10-PANVYR-04A-01D TARGET-10-PANWHJ-04A-01D TARGET-10-PANXPE-04A-01D TARGET-10-PANYGB-04A-01D TARGET-10-PANYYV-04A-01D TARGET-10-PANZPJ-04A-01D TARGET-10-PANZXC-04A-01D TARGET-10-PAPACP-04A-01D TARGET-10-PAPAGK-04A-01D TARGET-10-PAPCUR-04A-01D TARGET-10-PAPCVZ-04A-01D TARGET-10-PAPDJM-04A-01D TARGET-10-PAPDJM-09A-01D TARGET-10-PAPDKJ-04A-01D TARGET-10-PAPDUV-04A-01D TARGET-10-PAPDWT-04A-01D TARGET-10-PAPEAB-04A-01D TARGET-10-PAPECF-04A-01D TARGET-10-PAPGYC-04A-01D TARGET-10-PAPHEK-04A-01D TARGET-10-PAPIDY-04A-01D TARGET-10-PAPIJB-04A-01D TARGET-10-PAPISG-04A-01D TARGET-10-PAPJHB-04A-01D TARGET-10-PAPJHR-04A-01D TARGET-10-PAPJIB-04A-01D TARGET-10-PAPJXI-04A-01D TARGET-10-PAPLDL-04A-02D TARGET-10-PAPLDM-04A-01D TARGET-10-PAPLTZ-04

target SV wrong sample: PARJXH-DIAGNOSIS
target SV wrong sample: PARJXH-DIAGNOSIS
target SV wrong sample: PARJXH-DIAGNOSIS
target SV wrong sample: PARJXH-DIAGNOSIS
target SV wrong sample: PARJXH-DIAGNOSIS
target SV wrong sample: PARJXH-DIAGNOSIS
target SV wrong sample: PARJXH-DIAGNOSIS
target SV wrong sample: PARJXH-DIAGNOSIS
target SV wrong sample: PARJXH-DIAGNOSIS
target SV wrong sample: PARJXH-DIAGNOSIS
target SV wrong sample: PARJXH-DIAGNOSIS
target SV wrong sample: PARJXH-DIAGNOSIS
target SV wrong sample: PARJXH-DIAGNOSIS
pan-TARGET SV: dropped 42 lines of GRK1-CHST13
pan-TARGET SV: 0 lines of NBAS replaced in NBL
pan-TARGET SV: 0 lines of NBAS replaced in WT
Added 51 target NBL fusion
Added 107 target WT fusion


In [7]:
#3. NBL Cell lines
#dna_assay: wgs
#project: pedccl
#vorigin: somatic
#pmid: 29284669
#Tartan conserting output

import os

os.chdir('/research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/pcgp-target/nbl-cellline/')

!node collect.cnv.unpaired.js > raw/cnv.unpaired

!node parse.svCNV.2.js raw/cnv.unpaired raw/sv > svcnv




In [8]:
#4. Nalm6 SJBALL020016_C1 CNV
#sv.dux4IGH Credit: Nat Genet. 2016 May;48(5):569-74
#dna_assay: wgs
#project: pedccl
#vorigin: somatic
#pmid: ?


import os

os.chdir('/research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/pcgp-target/nalm6-cellline/')

!python3 parse_cnv.py /research/rgs01/resgen/prod/tartan/index/data/PCGP2/PCGP2/SJBALL020016_C1/WHOLE_GENOME/cnv-unpaired/SJBALL020016_C1_Single_CONSERTING_NoCREST_Mapability_100.txt

In [9]:
#4. Nalm16 SJHYPO010000_C213 CNV
#dna_assay: wgs
#project: pedccl
#vorigin: somatic
#pmid: ?

import os

os.chdir('/research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/pcgp-target/nalm16/')

!python3 parse_cnv.py /research/rgs01/resgen/legacy/gb_customTracks/tp/files/hg19/nalm16/wgs/cnv-unpaired/SJHYPO010000_C213_Single_CONSERTING_NoCREST_Mapability_100.txt


In [ ]:
#5. SCMC 
#Add 13 SCMC TALL 
#'SJALL018373_D1', 'SJALL018390_D1', 'SJALL040466_D1', 'SJALL040467_D1', 'SJALL040468_D1', 'SJALL040472_D1', 'SJALL043558_D1', 'SJALL043857_D1', 'SJALL043861_D1', 'SJALL043869_D1', 'SJTALL013797_D1', 'SJTALL013800_D1', 'SJTALL013801_D1'
#pmid: 31697823
#generate /research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/SCMC/2020_SCMC/scmc.cnvlohsv

#os.chdir('/research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/SCMC/2020_SCMC/')
#!python3 2020_SCMC_svcnvloh.py sv
#!python3 2020_SCMC_svcnvloh.py cnvloh sv


In [2]:
#6. merge pcgp+target+cell lines

import os

os.chdir('/research/rgs01/resgen/legacy/gb_customTracks/tp/hg19/pcgp-target/cnv-sv')


!cat ../../PCGP/cnv-sv/svcnvloh.annotated \
 ../../PCGP/cnv-sv/PCGP_fusion.annotated \
 ../../PCGP/itd/itd.bedj \
 ../../TARGET/cnv-sv/svcnv \
 ../../TARGET/cnv-sv/fusion \
 ../../TARGET/itd/itd.bedj \
 ../nbl-cellline/svcnv \
 ../nalm6-cellline/cnv \
 ../nalm6-cellline/sv.dux4IGH \
 ../nalm16/cnv \
 ../../SCMC/2020_SCMC/scmc.cnvlohsv \
 2020_pannbl.svcnv \
 > x

!sed -i.bak s/chr23/chrX/ x
!sed -i.bak s/chr24/chrY/ x
!sed -i.bak s/chrMT/chrM/ x

## get all samples
!node getsample.js x > samples

!sort -k1,1 -k2,2n x > ss
!cat samples ss > y
!bgzip y
!tabix -p bed -c'#sample' y.gz
#!mv y.gz pediatric.svcnv.hg19.gz
#!mv y.gz.tbi pediatric.svcnv.hg19.gz.tbi


GOT 3502 samples
[E::hts_idx_push] Region 1769099999..176999999 cannot be stored in a tbi index. Try using a csi index with min_shift = 14, n_lvls >= 6
tbx_index_build failed: y.gz
